## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [4]:
class myModel:
    def __init__(self):
        ####################
        # Declare model parameters
        ####################
        self.W1 = tf.Variable(shape=[28*28, 1000], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 1000],minval=-0.1, maxval=0.1))

        self.b1 = tf.Variable(shape=[1000], dtype=tf.float32, initial_value=tf.zeros(1000))

        self.W2 = tf.Variable(shape=[1000, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[1000, 10],minval=-0.1, maxval=0.1))

        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]

    def __call__(self, x):
        ####################
        # Implement model function
        ####################
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [5]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.1680284 ; accuracy 0.23055
epoch 1 : loss 2.1247077 ; accuracy 0.25593334
epoch 2 : loss 2.083572 ; accuracy 0.27971667
epoch 3 : loss 2.0443227 ; accuracy 0.30135
epoch 4 : loss 2.0067368 ; accuracy 0.32091665
epoch 5 : loss 1.9706458 ; accuracy 0.3405
epoch 6 : loss 1.9359204 ; accuracy 0.35985
epoch 7 : loss 1.9024585 ; accuracy 0.37908334
epoch 8 : loss 1.8701795 ; accuracy 0.39815
epoch 9 : loss 1.8390168 ; accuracy 0.41556665
epoch 10 : loss 1.8089138 ; accuracy 0.43205
epoch 11 : loss 1.7798214 ; accuracy 0.44726667
epoch 12 : loss 1.7516955 ; accuracy 0.46293333
epoch 13 : loss 1.7244966 ; accuracy 0.47716665
epoch 14 : loss 1.6981875 ; accuracy 0.49011666
epoch 15 : loss 1.6727333 ; accuracy 0.5024833
epoch 16 : loss 1.6481011 ; accuracy 0.51465
epoch 17 : loss 1.6242594 ; accuracy 0.5264
epoch 18 : loss 1.601178 ; accuracy 0.53826666
epoch 19 : loss 1.578828 ; accuracy 0.5487
epoch 20 : loss 1.5571814 ; accuracy 0.55883336
epoch 21 : loss 1.5362114 ; accuracy